In [3]:
from pyspark import SparkContext

In [4]:
sc = SparkContext(master="local", appName="TransformationsAndAccions")

In [5]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [6]:
rdd1.collect()

[1, 2, 3]

In [7]:
sc

<SparkContext master=local appName=TransformationsAndAccions>

In [8]:
!ls files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [9]:
path = "files/"

olympic_teams_RDD = sc.textFile(path+"paises.csv")\
    .map(lambda line: line.split(","))

In [10]:
olympic_teams_RDD.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [11]:
olympic_teams_RDD.map(lambda x: (x[2])).distinct().count()

231

In [12]:
olympic_teams_RDD.map(lambda x: (x[2],x[1]))\
    .groupByKey()\
    .mapValues(len)\
    .take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [13]:
# Creamos un RDD con la informacion de los equipos olimpicos

olympic_teams_RDD.map(lambda x: (x[2],x[1]))\
    .groupByKey()\
    .mapValues(list)\
    .take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [14]:
arg_teams = olympic_teams_RDD.filter(lambda x: "ARG" in x)

In [15]:
arg_teams.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [16]:
olympic_teams_RDD.count()

1185

In [17]:
olympic_teams_RDD.countApprox(20)

1185

In [18]:
!ls files/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [19]:
# Generamos dos RDDS apartir de archivos CSV

olympic_athlete_RDD = sc.textFile(path+"deportista.csv") \
    .map(lambda l: l.split(','))
olympic_athlete2_RDD = sc.textFile(path+"deportista2.csv") \
    .map(lambda l : l.split(','))

In [20]:
# Usamos join para unir dos RDDS que tienen la informacion de los atletas

olympic_athlete_RDD = olympic_athlete_RDD \
    .union(olympic_athlete2_RDD)

In [21]:
# Visualizamos 5 muestras del RDD creado

olympic_athlete_RDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [103]:
# .count() funcion para contar todos los registros en nuestro RDD

olympic_athlete_RDD.count()

135572

In [23]:
# .top(n) nos muestra los primeros n registros del RDD

olympic_athlete_RDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [24]:
olympic_teams_RDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [33]:
# Hacemos un join de los RDD que tiene la informacion de los deportistas con el RDD
# que contiene informacion de los equipos, apartir del ID de los deportistas

olympic_athlete_RDD.map(lambda l: (l[-1], l[:-1])) \
    .join(olympic_teams_RDD.map(lambda x: (x[0], x[2])))\
    .takeSample(False, 6, 25)

[('362', (['131505', 'Steven Woodburn', '1', '24', '185', '90'], 'FRA')),
 ('967', (['13626', 'Jill Brresen', '2', '22', '170', '57'], 'RSA')),
 ('482', (['44299', 'Gumundur Gumundsson', '1', '23', '174', '77'], 'ISL')),
 ('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('794', (['92442', 'Luis Paz Zoldan', '1', '19', '187', '82'], 'PER')),
 ('413', (['26822', 'Jared Mark Deacon', '1', '24', '185', '77'], 'GBR'))]

In [65]:
athlete_join_team_RDD = olympic_athlete_RDD.map(lambda l: [l[-1], l[:-1]]) \
    .join(olympic_teams_RDD.map(lambda x: [x[0], x[2]]))

In [26]:
# Creamos un RDD apartir de la informacion de las medallas ganadas por deportista

medals = sc.textFile(path+"resultados.csv")\
    .map(lambda l : l.split(","))

In [31]:
medals.top(3)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['99999', 'NA', '50604', '51', '568'],
 ['99998', 'NA', '50603', '47', '36']]

In [56]:
# Filtramos el RDD para eliminar valores nulos

medals_winned = medals.filter(lambda l : "NA" not in l[1])
medals_winned.top(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['99993', 'Silver', '50600', '39', '207']]

In [29]:
medals_winned.count()

39784

In [30]:
medals.count()

271117

In [64]:
athlete_join_team_RDD.top(2)

[('999', (['92679', 'Trygve Bjarne Pedersen', '1', '35', '0', '0'], 'NOR')),
 ('999', (['1144', 'Henrik Agersborg', '1', '47', '0', '0'], 'NOR'))]

In [83]:
# Juntamos el RDD que contiene la informacion de los atletas y sus equipos, con las
# medallas ganadas por equipos

athlete_join_team_RDD.map(lambda l: (l[1][0][0], (l[0],l[1][0][1:],l[1][1])))\
    .join(medals_winned.map(lambda x: (x[2],x[:2]+x[3:]))).top(5)
#     .filter(lambda x: 'Patimat' in x[1][0][1][0]).take(5)

[('99996',
  (('174', ['Sara Renner', '2', '21', '168', '63'], 'CAN'),
   ['199106', 'Silver', '46', '552'])),
 ('99992',
  (('399', ['Karlheinz Heinz Renneberg', '1', '25', '182', '87'], 'GER'),
   ['199089', 'Gold', '23', '114'])),
 ('9999',
  (('477', ['Ferenc Bene', '1', '19', '170', '70'], 'HUN'),
   ['18866', 'Gold', '25', '3'])),
 ('99988',
  (('302', ['Silke Renk Lange ', '2', '21', '173', '75'], 'GDR'),
   ['199082', 'Gold', '39', '70'])),
 ('99984',
  (('1117', ['Alphonse Renier', '1', '0', '0', '0'], 'BEL'),
   ['199075', 'Bronze', '2', '3']))]

In [84]:
# Juntamos el RDD que contiene la informacion de los atletas y sus equipos, con las
# medallas ganadas por equipos

athlete_team_medals = medals_winned.map(lambda l: (l[2], l[:2]+l[3:]))\
    .join(athlete_join_team_RDD.map(lambda x: (x[1][0][0],x[1])))

In [85]:
athlete_team_medals.take(5)

[('4',
  (['4', 'Gold', '2', '4'],
   (['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0'], 'SWE'))),
 ('16',
  (['41', 'Bronze', '50', '14'],
   (['16', 'Juhamatti Tapio Aaltonen', '1', '28', '184', '85'], 'FIN'))),
 ('20',
  (['61', 'Gold', '38', '32'],
   (['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85'], 'NOR'))),
 ('20',
  (['62', 'Bronze', '38', '33'],
   (['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85'], 'NOR'))),
 ('20',
  (['64', 'Silver', '40', '31'],
   (['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85'], 'NOR')))]

In [86]:
athlete_team_medals.filter(lambda x: 'Renner' in x[1][1][0][1]).take(5)

[('99996',
  (['199106', 'Silver', '46', '552'],
   (['99996', 'Sara Renner', '2', '21', '168', '63'], 'CAN')))]

In [88]:
# Creamos un diccionario con el valor de cada medalla

medal_values = {'Gold': 7,
                'Silver':5,
                'Bronze':4}

In [104]:
# Hacemos un nuevo RDD con la informacion del pais y medallas ganadas

countries_medals = athlete_team_medals\
    .map(lambda x: (x[1][1][-1], medal_values[x[1][0][1]]))

countries_medals.take(4)

[('SWE', 7), ('FIN', 4), ('NOR', 7), ('NOR', 4)]

In [105]:
from operator import add

In [106]:
# Realizamos una suma de puntaje por pais y ordenamos de mayor a menor

results_countries = countries_medals.reduceByKey((add))\
    .sortBy(lambda x: x[1], ascending=False)

In [107]:
results_countries.take(10)

[('USA', 32137),
 ('URS', 14834),
 ('GBR', 10925),
 ('GER', 10896),
 ('FRA', 9265),
 ('ITA', 8755),
 ('SWE', 8110),
 ('CAN', 7209),
 ('AUS', 6755),
 ('HUN', 6142)]

In [108]:
results_countries.filter(lambda x: "MEX" in x[0]).take(2)

[('MEX', 564)]

In [109]:
results_countries.filter(lambda x: "CHN" in x[0]).take(2)

[('CHN', 5362)]

In [110]:
sc.stop()